# Advanced STAC Exercise

Welcome to the advanced STAC exercise!

## Learning Objectives

By the end of this exercise, you will be able to:

1. Understand how STAC items provide direct access to individual satellite bands
2. Use rasterio to read data directly from remote URLs without downloading
3. Assess the performance benefits of Cloud Optimized GeoTIFFs
4. Build efficient workflows for satellite data analysis
5. Compare different approaches to accessing and processing satellite data

**Let's get started!**

## 1. Install Required Libraries

For this advanced exercise, we'll need additional libraries for raster data processing. Run the cell below to install all required packages.

In [ ]:
# Install required libraries
!pip install pystac-client rasterio matplotlib numpy

## 2. Understanding Direct Band Access in STAC

Unlike basic STAC workflows where we just discover metadata, advanced STAC usage involves:

### Direct URL Access
- Each STAC item contains URLs to individual bands
- These URLs point directly to the raster data (usually GeoTIFF files)
- No need to download entire datasets locally

### Cloud Optimized GeoTIFFs (COGs)
- Internal structure optimized for remote access
- Support for partial reads (only read what you need)
- Multiple resolution levels (overviews) for fast preview

### Performance Benefits
- Read only the spatial/spectral subset you need
- Reduced data transfer and processing time
- Scalable analysis without local storage constraints

Let's see this in action!

## 3. Connect to STAC Catalog and Find Data

First, let's connect to the EarthSearch STAC catalog and find some Sentinel-2 data over Amsterdam.

In [ ]:
# Import STAC Python Client library
import ____

In [ ]:
# Connect to EarthSearch STAC catalog
catalog = ____.Client.open("____")  # EarthSearch STAC API URL

# Define area of interest as Amsterdam, Netherlands
amsterdam_bbox = [____, ____, ____, ____]  # [west, south, east, north]

# Search for Sentinel-2 data
search = catalog.search(
    collections=["____"],             # Sentinel-2 L2A collection ID
    bbox=____,                            # Amsterdam, Netherlands
    datetime="2023-07-01/2023-07-31",     # July 2024
    max_items=____,                   # Limit to 5 items
    query={"eo:cloud_cover": {"lt": 20}}  # Less than 20% cloud cover
)

items = list(____.items())
print(f"Found {len(items)} items.")

# Select the first item for analysis
item = items[____]
print(f"Selected item: {item.id}")
print(f"Date: {item.____}")
print(f"Cloud cover: {item.____.get('____', 'N/A')}%")

## 4. Explore Available Assets

Let's examine what bands/assets are available in our STAC item and their direct access URLs.

In [ ]:
# Explore available assets in the STAC item
print("Available assets:")

for asset_key, asset in item.____.items():
    print(f"{asset_key}: {asset.title}")
    print(f"  Media type: {asset.____}")
    print(f"  URL: {asset.____}")

# Let's focus on some key bands for our analysis
bands_of_interest = ["____", "____", "____", "____"]  # Red, NIR, Green, Blue

print(f"We will work with these bands: {bands_of_interest}")
for band in bands_of_interest:
    if band in item.assets:
        url = item.assets[band].____
        print(f"  {band}: {url}")

## 5. Direct Band Access with Rasterio

We can use Rasterio library to open and read specific bands directly from their URLs. It also enables us to access remote data efficiently without downloading entire files, leveraging the power of COGs.

In [ ]:
# Import libraries
import ____  # For reading raster data
from rasterio.windows import Window

In [ ]:
# Get URLs for Red and NIR bands
red_url = item.assets["____"].href  # Red band
nir_url = item.assets["____"].href  # NIR band

print(f"Red band URL: {red_url}")
print(f"NIR band URL: {nir_url}")

# Open the bands with rasterio (without downloading)
with rasterio.open(red_url) as red_src:
    print(f"Red band info:")
    print(f"  Shape: {red_src.____}")
    print(f"  CRS: {red_src.____}")
    print(f"  Data type: {red_src.____}")
    
    # Read a small subset (first 100x100 pixels)
    window = Window(0, 0, ____, ____)  # Window size 100x100
    red_subset = red_src.read(1, ____=window)

with rasterio.open(nir_url) as nir_src:
    print(f"NIR band info:")
    print(f"  Shape: {nir_src.____}")
    print(f"  CRS: {nir_src.____}")
    print(f"  Data type: {nir_src.____}")
    
    # Read the same subset
    nir_subset = nir_src.read(1, ____=window)

print(f"Successfully read {window} subset from both bands.")
print(f"Red subset shape: {red_subset.____}")
print(f"NIR subset shape: {nir_subset.____}")

## 6. Performance Comparison - Full vs Partial Reads

Here, we compare the performance between full image reads and partial reads from COGs. The goal is to highlight how direct and selective data access can significantly improve processing time and efficiency.

In [ ]:
# Import libraries
import ____  # For timing operations

In [ ]:
print("Comparing performance...")

# Function to time operations
def time_operation(operation_name, operation_func):
    start_time = time.time()
    result = operation_func()
    end_time = time.____()
    duration = end_time - ____
    print(f"{operation_name}: {____:.2f} seconds")
    return result, duration

# Test 1: Read a small subset (512x512 pixels)
def read_subset():
    with rasterio.open(red_url) as src:
        window = Window(0, 0, ____, ____)  # Subset size 512x512
        return src.____(1, ____=window)

subset_data, subset_time = time_operation("Read 512x512 subset", read_subset)

# Test 2: Read entire band (be careful - this might be large!)
def read_full():
    with rasterio.open(red_url) as src:
        return src.____(____)  # Read entire band

full_data, full_time = time_operation("Read entire band", read_full)

# Test 3: Read with different overview levels
def read_overview():
    with rasterio.open(red_url) as src:
        return src.____(____, out_shape=(____, ____))  # 256x256 output shape for overview

overview_data, overview_time = time_operation("Read overview (lower res)", read_overview)

print("Notice how COGs allow fast access to different resolutions and subsets.")

## 7. NDVI Calculation Using Direct Band Access

We can calculate the Normalized Difference Vegetation Index (NDVI) using the directly accessed red and near-infrared bands. It will illustrate how cloud-optimized access enables seamless vegetation analysis without large downloads.

In [ ]:
print("Calculating NDVI using direct band access...")

# Define the area for analysis (smaller subset for faster processing)
analysis_window = ____(1000, 1000, ____, ____)  # Window size 800x800

# Read Red and NIR bands for the same window
with rasterio.open(red_url) as red_src:
    red_data = red_src.read(1, ____=analysis_window)

with rasterio.open(nir_url) as nir_src:
    nir_data = nir_src.read(1, ____=analysis_window)

# Calculate NDVI
# NDVI = (NIR - Red) / (NIR + Red)
ndvi = (nir_data - red_data) / (nir_data + red_data)

print(f"NDVI range: {ndvi.min():.3f} to {ndvi.max():.3f}")
print(f"NDVI shape: {ndvi.shape}")

Now, display the result.

Higher NDVI values (green) indicate healthier vegetation.

In [ ]:
# Import libraries
import ____ as plt

In [ ]:
# Create visualization
fig, axes = plt.____(1, 3, figsize=(15, 5))

# Plot Red band
axes[0].____(red_data, cmap='Reds')
axes[0].set_title('Red Band')
axes[0].axis('off')

# Plot NIR band
axes[1].____(nir_data, cmap='RdYlGn')
axes[1].set_title('NIR Band')
axes[1].axis('off')

# Plot NDVI
ndvi_plot = axes[2].____(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
axes[2].set_title('NDVI')
axes[2].axis('off')
plt.colorbar(ndvi_plot, ax=axes[2], shrink=0.6)

plt.tight_layout()
plt.____()

## 8. Multi-Band RGB Composite

Similarly, we can create an RGB composite image by combining multiple bands into a single visual representation. True color composite shows Amsterdam as it would appear to the human eye.

In [ ]:
# Import libraries
import ____ as np

In [ ]:
print("Creating RGB composite using direct band access...")

# Get URLs for RGB bands
red_url = item.assets["____"].href    # Red band
green_url = item.assets["____"].href  # Green band
blue_url = item.assets["____"].href   # Blue band

# Define window for RGB composite
rgb_window = ____(2000, 2000, 1000, 1000)

# Read all three bands
with rasterio.open(red_url) as red_src:
    red_band = red_src.____(1, ____=rgb_window)

with rasterio.open(____) as green_src:
    green_band = green_src.____(1, ____=rgb_window)

with rasterio.open(____) as blue_src:
    blue_band = blue_src.____(1, ____=rgb_window)

# Normalize for visualization (scale to 0-255)
def normalize_band(band, percentile=98):
    # Use percentile scaling for better visualization
    upper = np.percentile(band, percentile)
    lower = np.percentile(band, 2)
    return np.clip((band - lower) / (upper - lower) * 255, 0, 255).astype(np.uint8)

# Normalize each band
rgb_normalized = np.stack([
    ____(red_band),
    ____(green_band),
    ____(blue_band)
], axis=____)  # Axis 0 for band stacking

# Display RGB composite
plt.____(figsize=(10, 10))
plt.imshow(rgb_normalized.transpose(____, ____, ____))  # Transpose for matplotlib
plt.title('True Color RGB Composite', fontsize=14)
plt.axis('off')
plt.show()

## 9. Efficient Workflow for Multiple Items Analysis

Finally, we can extend the workflow to handle multiple STAC items at once. This will enable us to automate and scale the analysis process for broader spatial or temporal studies using the same efficient techniques.

In [ ]:
print("Creating NDVI time series...")

# Search for more items over a longer time period
time_series_search = catalog.search(
    ____=["____"],                 # Sentinel-2 L2A collection
    bbox=____,                     # Amsterdam, Netherlands
    ____="____/____",              # May to August 2024
    max_items=____,                # Maximum 10 items
    ____={"____": {"____": ____}}  # Less than 30% cloud cover
)

time_series_items = list(time_series_search.____())
print(f"Found {len(____)} items for time series analysis.")

# Define a small area for consistent analysis
analysis_window = Window(2500, 2500, 200, 200)

# Function to calculate mean NDVI for an item
def calculate_mean_ndvi(item, window):
    try:
        red_url = item.assets["____"].____
        nir_url = item.assets["____"].____
        
        with rasterio.open(red_url) as red_src:
            red_data = red_src.____(____, ____=window)
        
        with rasterio.open(nir_url) as nir_src:
            nir_data = nir_src.____(____, ____=window)
        
        # Calculate NDVI
        ndvi = (nir_data - red_data) / (nir_data + red_data + ____)  # Add small value to avoid division by zero
        
        # Return mean NDVI
        return np.nanmean(ndvi)
    
    except Exception as e:
        print(f"Error processing item {item.id}: {e}")
        return None

# Process each item to get time series data
dates = []
ndvi_values = []

for item in time_series_items:
    mean_ndvi = calculate_mean_ndvi(item, analysis_window)
    if mean_ndvi is not None:
        dates.____(item.____)
        ndvi_values.____(____)
        print(f"{item.datetime.____('%Y-%m-%d')}: NDVI = {____:.3f}")

print(f"Processed {len(ndvi_values)} valid observations")
print(f"NDVI range: {min(ndvi_values):.3f} to {max(ndvi_values):.3f}")
print(f"All data accessed directly from URLs, no local downloads required.")

Now, display the results.

In [ ]:
# Create time series plot
plt.figure(figsize=(12, 6))
plt.plot(____, ____, 'go-', linewidth=2, markersize=8)
plt.title('NDVI Time Series', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Mean NDVI')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.____()

# Conclusion

Congratulations, you have successfully completed the advanced STAC exercise!

## What You've Learned

1. **STAC + COG = Powerful Combination**: STAC provides discovery, COGs enable efficient access
2. **Subset Reading**: Only read the data you need - saves time and bandwidth
3. **No Downloads Required**: Process satellite data without local storage
4. **Scalable Analysis**: Techniques work for small areas or global analysis
5. **Multiple Resolutions**: Use overviews for quick analysis, full resolution for detailed work

**Well done! You're now one step closer to build efficient, scalable satellite data analysis workflows!**